# Homework 4

Implementing the RI EKF, based on gyroscope and accelerometer data

In [54]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
class RIEKF:
    def __init__(self, model):
        """
        Construct an instance of this class

        Parameters
        ----------
        model:     system and noise models
        """
        
        self.dt = 0.1

# State estimation filter class

        self.R = model.R # measurement noise covariance
        self.Q = model.Q # process noise covariance
        if model is None:
            raise ValueError("model cannot be None")

        self.H = model.H
        self.b = model.b
        if model.R is None:
            raise ValueError("model.R cannot be None")

        self.A = model.A

        if model.Q is None:
            raise ValueError("model.Q cannot be None")
        self.state = model.state
        self.P = model.P
        self.state_transition_matrix = self.get_state_transition_matrix(model.A, self.dt)
        self.process_model = model.f

    def wedge(self, x):
        """
        Wedge operation for se(2) to put an R^3 vector to the Lie algebra basis

        Parameters
        ----------
        x: ndarray
            R^3 vector

        Returns
        -------
        xhat: ndarray
            Lie algebra basis
        """
        # wedge operation for se(2) to put an R^3 vector to the Lie algebra basis
        G1 = np.array([[0, -1, 0],
                       [1, 0, 0],
                       [0, 0, 0]])  # omega
        G2 = np.array([[0, 0, 1],
        
                       [0, 0, 0],
                       [-1, 0, 0]])  # v_1
        G3 = np.array([[0, 0, 0],
                       [0, 0, -1],
                       [0, 1, 0]])  # v_2
        xhat = G1 * x[0] + G2 * x[1] + G3 * x[2]
        return xhat

    def predict(self, u, dt):
        """
        Propagate the state and error covariance

        Parameters
        ----------
        u: ndarray
            control input
        dt: float
            time step

        Returns
        -------
        None
        """
        self.state_transition_matrix = self.get_state_transition_matrix(self.A, dt)

        # Propagate the state and error covariance
        self.P = self.state_transition_matrix@ self.P@ self.state_transition_matrix.T + self.state @ self.Q @ self.state.T

        # print("state transition matrix: ", self.state_transition_matrix)        
        self.state = self.process_model(self.state, self.wedge(u), dt)
        
    
    def update(self, Y):
        """
        Update the state and error covariance

        Parameters
        ----------
        Y: ndarray
            measurement

        Returns
        -------
        None
        """

        # Update the state and error covariance
        # Innovation covariance
        state_inv = self.state
        N = state_inv @ self.R @ state_inv.T
        N = N[0:2, 0:2] # as H is 2x3, so make N 2x2
        S = self.H @ self.P @ self.H.T + N

        # Gain
        gain = self.P @ self.H.T @ np.linalg.inv(S)
        Y = Y[0:2]
        state_inv = state_inv[0:2, 0:2]

        

        # print("Shapes: ", "Y shape : ", Y.shape, "state_inv shape : ", state_inv.shape,  "R shape : ", self.R.shape, "N shape : ", N.shape, "S shape : ", S.shape, "gain shape : ", gain.shape)
        print("gain: ", gain)
        print("Y: ", Y)
        print("state_inv: ", state_inv)
        print("S: ", S)
        print("N: ", N)

        # Update State and Error Covariance for Right Invariant method
        self.state =self.wedge(np.exp(gain@(state_inv@Y - self.b)))@self.state
        self.P = (np.eye(3) - gain@self.H)@self.P@(np.eye(3) - gain@self.H).T + gain@N@gain.T

    def get_state_transition_matrix(self, A, dt):
        return np.exp(A*dt)

class Model:
    def __init__(self):
        """
        Compute the state transition matrix

        Parameters
        ----------
        A: ndarray
            error dynamics matrix
        dt: float
            time step

        Returns
        -------
        expA: ndarray
            state transition matrix
        """
        self.R = np.eye(3)*0.5 # measurement noise covariance , random choice
        self.Q = np.eye(3)*0.4 # process noise covariance,  random choice
        
        # System and noise models
        self.A = np.zeros((3, 3)) # error dynamics matrix (A_t^r)
        self.H = np.zeros((3, 3)) # measurement error matrix HE = E^b
        self.H[0,1] = 9.81 
        self.H[1,2] = -9.81
        self.H = self.H[0:2] # last row is zero so remove it from H, then H is 2x3

        self.b = np.zeros((3, 1))
        self.b[2] = 9.8
        self.b = self.b[0:2] # last row is zero for H, so remove it from b, for EKF calculation

        self.P = np.eye(3) # error covariance
        self.state = np.eye(3) # state


    def f(self, x, u, dt):
        # Process model (descritized)(Xdot = Xu / Rdot = R*omega)
        return x @ np.exp(u*dt)



In [56]:
data_dir = "data.csv"

Y = pd.read_csv(os.path.join(data_dir, "a.csv")).to_numpy()
DT = pd.read_csv(os.path.join(data_dir, "dt.csv")).to_numpy()

EULER = pd.read_csv(os.path.join(data_dir, "euler_gt.csv")).to_numpy()
G = pd.read_csv(os.path.join(data_dir, "gravity.csv")).to_numpy()
O = pd.read_csv(os.path.join(data_dir, "omega.csv")).to_numpy()



In [52]:
Y

array([[-1.87956107,  0.85779035,  9.70845318],
       [-1.78034461,  0.92175812,  9.62155056],
       [-1.76939213,  0.97230649,  9.49869442],
       ...,
       [ 0.45520639, -0.84562975, 10.0117178 ],
       [ 0.47471404, -0.68404317,  9.91699123],
       [ 0.34381217, -0.4562003 ,  9.83192825]])

In [ ]:

model = Model()
reiefk = RIEKF(model)

states = []

# Loop over the data
for i in range(0, 10):
    dt = DT[i]
    u = O[i]
    y = Y[i].reshape(-1,1)

    # print("dt : ",dt)
    dt = 0.01
    # print("u: ", u)
    # print(reiefk.wedge(u))
    print(np.exp(reiefk.wedge(u)))
    reiefk.predict(u, dt)
    print("predicted state: ", reiefk.state)
    reiefk.update(y)

    print("updated state: ", reiefk.state)

    states.append(reiefk.state)

states = np.array(states)

[[1.         1.00661038 0.99697437]
 [0.99343303 1.         0.97893755]
 [1.00303481 1.02151562 1.        ]]
predicted state:  [[1.         1.00006589 0.9999697 ]
 [0.99993412 1.         0.99978715]
 [1.0000303  1.0002129  1.        ]]
gain:  [[ 0.04778269 -0.04778306]
 [ 0.09825191 -0.00368486]
 [ 0.0036845  -0.09825233]]
Y:  [[-1.87956107]
 [ 0.85779035]]
state_inv:  [[1.         1.00006589]
 [0.99993412 1.        ]]
S:  [[ 328.70277559 -287.20842157]
 [-287.20842157  328.70246129]]
N:  [[1.50003559 1.49987843]
 [1.49987843 1.49972129]]
updated state:  [[-0.0920035  -0.0919036  -0.09188404]
 [-0.10147211 -0.10160733 -0.10146903]
 [ 0.19346305  0.1934758   0.19332869]]
[[1.         0.98176628 1.01777456]
 [1.01857236 1.         0.95507216]
 [0.98253586 1.04704131 1.        ]]
predicted state:  [[-0.27579187 -0.27581646 -0.27576511]
 [-0.3045493  -0.30457646 -0.30451966]
 [ 0.5802691   0.58032084  0.58021272]]
gain:  [[ 0.09643022 -0.00545628]
 [ 0.09958224 -0.00230057]
 [ 0.00260008 -

: 